In [1]:
import string
import time
import numpy as np

negative_file_adr = '../dataset/rt-polarity.neg'
positive_file_adr = '../dataset/rt-polarity.pos'

# reading dataset
start = time.time()
with open(negative_file_adr, 'r') as neg_file:
    neg_lines = neg_file.readlines()
    
    
with open(positive_file_adr, 'r') as pos_file:
    pos_lines = pos_file.readlines()
end = time.time()

print(f'elapsed time: {end - start}')

neg_test_dataset = np.random.choice(neg_lines, int(np.floor(0.1 * len(neg_lines))), replace=False)
pos_test_dataset = np.random.choice(pos_lines, int(np.floor(0.1 * len(pos_lines))), replace=False)

neg_lines = [n for n in neg_lines if n not in neg_test_dataset]
pos_lines = [n for n in pos_lines if n not in pos_test_dataset]

print(f'number of lines in neg_lines: {len(neg_lines)}')
print(f'example of line in neg_lines: {neg_lines[0]}')
print(f'number of lines in pos_lines: {len(pos_lines)}')
print(f'example of line in pos_lines: {pos_lines[0]}')

print(len(neg_test_dataset))
print(len(pos_test_dataset))


elapsed time: 0.0059967041015625
number of lines in neg_lines: 4798
example of line in neg_lines: it's so laddish and juvenile , only teenage boys could possibly find it funny . 

number of lines in pos_lines: 4798
example of line in pos_lines: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 

533
533


In [2]:
def clean(lines):
    start_symbol = '<s>'
    end_symbol = '</s>'
    cleaned_up = []

    for line in lines:
        line = line.translate({ord(x): ' ' for x in string.punctuation})
        line = line.translate({ord(x): ' ' for x in line if x not in string.printable})
        line = line.split()
        line.insert(0, start_symbol)
        line.append(end_symbol)
        cleaned_up.append(line)
        
    return cleaned_up

print(f'example of cleaned line: {clean(neg_lines)[0]}')

example of cleaned line: ['<s>', 'it', 's', 'so', 'laddish', 'and', 'juvenile', 'only', 'teenage', 'boys', 'could', 'possibly', 'find', 'it', 'funny', '</s>']


In [3]:
from collections import OrderedDict
def make_unigram_dictionary(cleaned_lines):
    dictionary = {}
    for line in cleaned_lines:
        for word in line:
            if word in dictionary:
                dictionary[word] += 1
            else:
                dictionary[word] = 1
    return OrderedDict(sorted(dictionary.items()))

def make_bigram_dictionary(cleaned_lines):
    dictionary = {}
    for line in cleaned_lines:
        for i in range(len(line) - 1):
            first_word  = line[i]
            second_word = line[i+1]
            combination = first_word + ' ' + second_word
            if combination in dictionary:
                dictionary[combination] += 1
            else:
                dictionary[combination] = 1
                
    return OrderedDict(sorted(dictionary.items()))

In [4]:
start = time.time()
neg_unigram_dict = make_unigram_dictionary(clean(neg_lines))
neg_bigram_dict = make_bigram_dictionary(clean(neg_lines))
pos_unigram_dict = make_unigram_dictionary(clean(pos_lines))
pos_bigram_dict = make_bigram_dictionary(clean(pos_lines))
end = time.time()

print(f'elapsed time: {end - start}')

print('\nfirst 10 records in neg_unigram: (sorted by alphabetical order)\n')
counter = 0
for key, value in neg_unigram_dict.items():
    print(f'  {key}: {value}')
    counter += 1
    if counter == 10:
        break

print('\nfirst 10 records in neg_bigram: (sorted by alphabetical order)\n')
counter = 0
for key, value in neg_bigram_dict.items():
    print(f'  {key}: {value}')
    counter += 1
    if counter == 10:
        break

elapsed time: 0.8503944873809814

first 10 records in neg_unigram: (sorted by alphabetical order)

  00: 1
  000: 2
  007: 2
  1: 6
  10: 14
  100: 7
  101: 3
  102: 1
  103: 1
  104: 1

first 10 records in neg_bigram: (sorted by alphabetical order)

  00 </s>: 1
  000 leagues: 1
  000 times: 1
  007 cannot: 1
  007 clone: 1
  1 2: 3
  1 </s>: 1
  1 but: 1
  1 is: 1
  10 000: 1


In [5]:
# for testing
s = 0
for key, value in neg_unigram_dict.items():
    s += value
print(s)

t = 0
for line in clean(neg_lines):
    t += len(line)
print(t)

assert t == s, 'something is wrong in making unigram dictionary'

102484
102484


In [72]:
import math
def probablity_of_sentense(sentence, unigram_dict, bigram_dict, number_of_words, scalars_list, log=False):
    
    # without logarithm
    def unigram_probablity(word, unigram_dict, number_of_words):
        return unigram_dict[word] / number_of_words if word in unigram_dict else 0
    
    def bigram_probablity(first_word, second_word, bigram_dict, unigram_dict, number_of_words):
        combination = first_word + ' ' + second_word
        
        count_combination = 0
        try:
            count_combination = bigram_dict[combination]
        except:
            pass
        
        count_first_word = 0
        try: 
            count_first_word = unigram_dict[first_word]
        except:
            pass
        
        return (count_combination + 1) / (count_first_word + number_of_words)
    
    
#      # with logarithm
#     def unigram_probablity(word, unigram_dict, number_of_words):
#         return math.log10(unigram_dict[word] / number_of_words) if word in unigram_dict else 0
    
#     def bigram_probablity(first_word, second_word, bigram_dict, unigram_dict, number_of_words):
#         combination = first_word + ' ' + second_word
        
#         count_combination = 0
#         try:
#             count_combination = bigram_dict[combination]
#         except:
#             pass
        
#         count_first_word = 0
#         try: 
#             count_first_word = unigram_dict[first_word]
#         except:
#             pass
        
#         return math.log10((count_combination + 1) / (count_first_word + number_of_words))

    
    def interpolated_bigram_probablity(first_word, second_word, uigram_dict, bigram_dict, number_of_words, scalars_list):
        
        l3 = scalars_list[0]
        l2 = scalars_list[1]
        l1 = scalars_list[2]
        e  = scalars_list[3]
        
        return l3 * bigram_probablity(first_word, second_word, bigram_dict, unigram_dict, number_of_words) + \
               l2 * unigram_probablity(second_word, unigram_dict, number_of_words) + \
               l1 * e
    
    sentence = clean([sentence])[0]

    total_probablity = unigram_probablity(sentence[0], unigram_dict, number_of_words)
    if log: print(f'w0  : {sentence[0]:<47}, prob:{total_probablity:<30}')
    
    for i in range(1, len(sentence) - 1):
        first_word = sentence[i]
        second_word = sentence[i+1]
        p = interpolated_bigram_probablity(first_word, second_word, unigram_dict, bigram_dict, number_of_words, scalars_list)
        total_probablity *= p
        
        if log: print(f'w{i:<3}: {first_word:<20}, w{i+1:<3}:{second_word:<20}, prob:{p:<30}')
        
    if log: print(f'FINAL PROBABILITY: {total_probablity:}\n')
    return total_probablity

In [7]:
def detect_language(sentence, language0, language1, scalars_list, log=False):
    
    if log: print('in Lang0')
    language0_porbability = probablity_of_sentense(sentence, language0['unigram_dict'], language0['bigram_dict'],
                                                   language0['words_number'],scalars_list, log=log)
    if log: print('in Lang1')
    language1_porbability = probablity_of_sentense(sentence, language1['unigram_dict'], language1['bigram_dict'],
                                                   language1['words_number'],scalars_list, log=log)

    
    if language0_porbability == 0 and language1_porbability != 0:
        return 1
    elif language0_porbability != 0 and language1_porbability == 0:
        return 0
    elif language0_porbability == 0 and language1_porbability == 0:
        return -1
    else :
        if language0_porbability > language1_porbability:
            return 0
        else:
            return 1
            
#     if language0_porbability > language1_porbability:
#         return 0
#     elif language0_porbability < language1_porbability:
#         return 1
#     else:
#         return -1


In [17]:
def accuracy_test(language0_data, language1_data, scalars_list):
    log = False

    successful_detection = 0
    for positive_sentence in language0_data['test_dataset']:
        detected = detect_language(positive_sentence, language0_data, language1_data, scalars_list, log=log)
        if log: print(f'detected : {detected}')
        if detected == 0:
            successful_detection += 1

    positive_dataset_accuracy = successful_detection / len(pos_test_dataset)

    successful_detection = 0
    for negative_sentence in language1_data['test_dataset']:
        detected = detect_language(negative_sentence, language0_data, language1_data ,scalars_list, log=log)
        if log: print(f'detected : {detected}')
        if detected == 1:
            successful_detection += 1

    negative_dataset_accuracy = successful_detection / len(neg_test_dataset)
    
    return positive_dataset_accuracy, negative_dataset_accuracy

In [73]:
import pprint
positive_words_number = 0
for key, value in pos_unigram_dict.items():
    positive_words_number += value

negative_words_number = 0
for key, value in neg_unigram_dict.items():
    negative_words_number += value
    
language0_data ={
    'unigram_dict': neg_unigram_dict,
    'bigram_dict': neg_bigram_dict,
    'words_number': negative_words_number,
    'test_dataset': neg_test_dataset
}

language1_data ={
    'unigram_dict': pos_unigram_dict,
    'bigram_dict': pos_bigram_dict,
    'words_number': positive_words_number,
    'test_dataset': pos_test_dataset
}

scalars = np.array([0.9, 0.09, 0.01, 0.01])
pos_ac, neg_ac = accuracy_test(language0_data, language1_data, scalars / sum(scalars))
print(pos_ac)
print(neg_ac)

test_dictionary = {}
for i in range(500):
    scalars = np.random.random(3)
    scalars = scalars / sum(scalars)
    scalars = np.append(scalars, np.random.random(1)[0])
    test_dictionary[(i, tuple(scalars))] = None
    print(f'testcase: {i:<4},{scalars}, SUM[:3]: {sum(scalars[:3])}')

for key in test_dictionary:
    pos_ac, neg_ac = accuracy_test(language0_data, language1_data, key[1])
    test_dictionary[key] = (pos_ac, neg_ac)

0.7185741088180112
0.7204502814258912
testcase: 0   ,[0.63882378 0.11273665 0.24843956 0.82537923], SUM[:3]: 1.0
testcase: 1   ,[0.17110172 0.40983288 0.4190654  0.33304707], SUM[:3]: 1.0
testcase: 2   ,[0.26531645 0.17094179 0.56374177 0.19489309], SUM[:3]: 1.0
testcase: 3   ,[0.19401555 0.73679833 0.06918612 0.23507794], SUM[:3]: 1.0
testcase: 4   ,[0.16209477 0.38289585 0.45500938 0.96062996], SUM[:3]: 1.0
testcase: 5   ,[0.20122311 0.31124201 0.48753488 0.81003499], SUM[:3]: 1.0
testcase: 6   ,[0.59057482 0.36261394 0.04681124 0.64377366], SUM[:3]: 1.0
testcase: 7   ,[0.36150258 0.34948962 0.2890078  0.71275938], SUM[:3]: 1.0
testcase: 8   ,[0.14432471 0.27237655 0.58329874 0.06755628], SUM[:3]: 1.0
testcase: 9   ,[0.53407036 0.34304225 0.12288738 0.12736995], SUM[:3]: 1.0
testcase: 10  ,[0.7274113  0.13060124 0.14198746 0.80422643], SUM[:3]: 1.0
testcase: 11  ,[0.08394662 0.35698324 0.55907015 0.86775555], SUM[:3]: 1.0
testcase: 12  ,[0.40323631 0.2051     0.39166369 0.07623757], 

testcase: 492 ,[0.24167218 0.54812387 0.21020394 0.86597837], SUM[:3]: 1.0
testcase: 493 ,[0.48372646 0.13132646 0.38494708 0.47903858], SUM[:3]: 1.0
testcase: 494 ,[0.44149431 0.51406973 0.04443596 0.98497226], SUM[:3]: 1.0
testcase: 495 ,[0.50833368 0.04392407 0.44774225 0.45434408], SUM[:3]: 1.0
testcase: 496 ,[0.51859105 0.25259398 0.22881497 0.70194253], SUM[:3]: 1.0
testcase: 497 ,[0.47441879 0.15359974 0.37198147 0.22011978], SUM[:3]: 1.0
testcase: 498 ,[0.33585861 0.43115385 0.23298753 0.27417023], SUM[:3]: 1.0000000000000002
testcase: 499 ,[0.43215307 0.37437571 0.19347122 0.76771387], SUM[:3]: 1.0


In [74]:
with open('./results.txt', 'w') as result_file:
    c = {k: v for k, v in sorted(test_dictionary.items(), key=lambda item: sum(item[1]), reverse=True)}
    for key, v in c.items():
        k = list(key[1])
        p = 4
        line = f'testcase: {key[0]:<4}[l3: {np.round(k[0], 4):<6}][l2: {np.round(k[1], 4):<6}][l1: {np.round(k[2], 4):<6}][e: {np.round(k[3], 4):<6}] ==> [positive accuracy: {np.round(v[0], 4):<6}][negative accuracy: {np.round(v[1], 4):<6}]'
        print(line)
        result_file.write(line + '\n')

testcase: 479 [l3: 0.2922][l2: 0.7041][l1: 0.0038][e: 0.0161] ==> [positive accuracy: 0.7711][negative accuracy: 0.7186]
testcase: 309 [l3: 0.4765][l2: 0.4605][l1: 0.0629][e: 0.0094] ==> [positive accuracy: 0.6735][negative accuracy: 0.7073]
testcase: 201 [l3: 0.0486][l2: 0.4648][l1: 0.4866][e: 0.001 ] ==> [positive accuracy: 0.6698][negative accuracy: 0.7073]
testcase: 109 [l3: 0.3257][l2: 0.5637][l1: 0.1106][e: 0.0071] ==> [positive accuracy: 0.6492][negative accuracy: 0.7148]
testcase: 156 [l3: 0.6303][l2: 0.3632][l1: 0.0065][e: 0.1087] ==> [positive accuracy: 0.6379][negative accuracy: 0.7186]
testcase: 229 [l3: 0.1742][l2: 0.7828][l1: 0.043 ][e: 0.0339] ==> [positive accuracy: 0.6116][negative accuracy: 0.7355]
testcase: 186 [l3: 0.4177][l2: 0.4363][l1: 0.146 ][e: 0.0111] ==> [positive accuracy: 0.5685][negative accuracy: 0.7561]
testcase: 82  [l3: 0.9013][l2: 0.0048][l1: 0.0939][e: 0.2737] ==> [positive accuracy: 0.6473][negative accuracy: 0.6604]
testcase: 468 [l3: 0.44  ][l2: 0

In [80]:
test_dictionary_relaxed = {}
for i in range(5000):
    scalars = np.random.random(2)
    scalars = np.append(scalars, 0.001) # l1
    scalars = scalars / sum(scalars)
    scalars = np.append(scalars, 0.001) # e
    test_dictionary_relaxed[(i, tuple(scalars))] = None
    print(f'testcase: {i:<4},{scalars}, SUM[:3]: {sum(scalars[:3])}')

for key in test_dictionary_relaxed:
    pos_ac, neg_ac = accuracy_test(language0_data, language1_data, key[1])
    test_dictionary_relaxed[key] = (pos_ac, neg_ac)

testcase: 0   ,[0.29538627 0.70229424 0.00231949 0.001     ], SUM[:3]: 1.0
testcase: 1   ,[0.15599944 0.84276656 0.001234   0.001     ], SUM[:3]: 0.9999999999999999
testcase: 2   ,[0.42166347 0.57705542 0.00128111 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 3   ,[0.76933926 0.22856781 0.00209293 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 4   ,[0.49702486 0.50231569 0.00065944 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 5   ,[0.39344433 0.60571489 0.00084078 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 6   ,[0.50372983 0.49565112 0.00061905 0.001     ], SUM[:3]: 1.0
testcase: 7   ,[0.15637478 0.84265855 0.00096668 0.001     ], SUM[:3]: 1.0
testcase: 8   ,[0.01134617 0.98624504 0.00240879 0.001     ], SUM[:3]: 1.0
testcase: 9   ,[0.78467801 0.21325991 0.00206209 0.001     ], SUM[:3]: 1.0
testcase: 10  ,[0.8486941  0.14947871 0.00182719 0.001     ], SUM[:3]: 1.0
testcase: 11  ,[0.76571103 0.23240671 0.00188226 0.001     ], SUM[:3]: 1.0
testcase: 12  ,[0.6617531

testcase: 369 ,[0.28532528 0.7138735  0.00080123 0.001     ], SUM[:3]: 1.0
testcase: 370 ,[0.91499094 0.08361798 0.00139107 0.001     ], SUM[:3]: 1.0
testcase: 371 ,[0.44021645 0.5591099  0.00067365 0.001     ], SUM[:3]: 1.0
testcase: 372 ,[0.59805084 0.40126211 0.00068705 0.001     ], SUM[:3]: 1.0
testcase: 373 ,[0.26382525 0.73458804 0.00158671 0.001     ], SUM[:3]: 1.0
testcase: 374 ,[0.45398935 0.5452984  0.00071225 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 375 ,[0.66231018 0.33605913 0.00163069 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 376 ,[0.46161    0.53782205 0.00056795 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 377 ,[0.88505851 0.11240257 0.00253892 0.001     ], SUM[:3]: 1.0
testcase: 378 ,[0.01264424 0.98610275 0.00125302 0.001     ], SUM[:3]: 1.0
testcase: 379 ,[0.51193234 0.48705016 0.0010175  0.001     ], SUM[:3]: 1.0
testcase: 380 ,[0.28344444 0.71461484 0.00194072 0.001     ], SUM[:3]: 1.0
testcase: 381 ,[0.43608273 0.56307326 0.00084401 0.001 

testcase: 724 ,[0.4970659  0.50197719 0.00095691 0.001     ], SUM[:3]: 1.0
testcase: 725 ,[0.32208633 0.67713953 0.00077414 0.001     ], SUM[:3]: 1.0
testcase: 726 ,[0.14965698 0.8494894  0.00085362 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 727 ,[0.54161195 0.45746019 0.00092786 0.001     ], SUM[:3]: 1.0
testcase: 728 ,[0.79962087 0.19851662 0.00186251 0.001     ], SUM[:3]: 1.0
testcase: 729 ,[0.57733778 0.4220835  0.00057872 0.001     ], SUM[:3]: 1.0
testcase: 730 ,[0.42849617 0.56942142 0.00208241 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 731 ,[0.22620839 0.77283117 0.00096043 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 732 ,[0.65901902 0.34031566 0.00066532 0.001     ], SUM[:3]: 1.0
testcase: 733 ,[0.51835861 0.48107863 0.00056276 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 734 ,[0.41499429 0.58437673 0.00062899 0.001     ], SUM[:3]: 1.0
testcase: 735 ,[0.50847248 0.49060087 0.00092664 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 736 ,[0.0187736

testcase: 1179,[0.32983618 0.66930405 0.00085978 0.001     ], SUM[:3]: 1.0
testcase: 1180,[0.46358067 0.53583891 0.00058041 0.001     ], SUM[:3]: 1.0
testcase: 1181,[0.32700638 0.67173173 0.00126189 0.001     ], SUM[:3]: 1.0
testcase: 1182,[0.88257098 0.11556886 0.00186016 0.001     ], SUM[:3]: 1.0
testcase: 1183,[0.60287524 0.39611209 0.00101267 0.001     ], SUM[:3]: 1.0
testcase: 1184,[0.81854624 0.18036845 0.0010853  0.001     ], SUM[:3]: 1.0
testcase: 1185,[0.50497795 0.49416993 0.00085212 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 1186,[0.3527594  0.64637698 0.00086362 0.001     ], SUM[:3]: 1.0
testcase: 1187,[0.30465054 0.6937541  0.00159536 0.001     ], SUM[:3]: 1.0
testcase: 1188,[0.4454147  0.55334396 0.00124133 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 1189,[0.46154628 0.53787094 0.00058278 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 1190,[0.27374364 0.72279967 0.00345668 0.001     ], SUM[:3]: 1.0
testcase: 1191,[0.48877383 0.51032165 0.00090453 0.001 

testcase: 1623,[0.35726544 0.64140559 0.00132897 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 1624,[0.94734953 0.05093823 0.00171225 0.001     ], SUM[:3]: 1.0
testcase: 1625,[0.046325   0.95088089 0.00279411 0.001     ], SUM[:3]: 1.0
testcase: 1626,[0.81391872 0.1852051  0.00087618 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 1627,[0.36076755 0.63857568 0.00065677 0.001     ], SUM[:3]: 1.0
testcase: 1628,[0.26489288 0.7335764  0.00153072 0.001     ], SUM[:3]: 1.0
testcase: 1629,[0.12251095 0.87645259 0.00103646 0.001     ], SUM[:3]: 1.0
testcase: 1630,[0.58649611 0.41218295 0.00132094 0.001     ], SUM[:3]: 1.0
testcase: 1631,[0.65824165 0.33858756 0.00317079 0.001     ], SUM[:3]: 1.0
testcase: 1632,[0.62990411 0.36887155 0.00122434 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 1633,[0.80228265 0.19659158 0.00112577 0.001     ], SUM[:3]: 1.0
testcase: 1634,[0.50783981 0.49137041 0.00078978 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 1635,[0.50336963 0.49609377 0.

...

testcase: 2920,[0.54409257 0.45502346 0.00088397 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 2921,[0.52774092 0.47112102 0.00113806 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 2922,[0.58814417 0.41123846 0.00061737 0.001     ], SUM[:3]: 1.0
testcase: 2923,[0.46003465 0.5388516  0.00111375 0.001     ], SUM[:3]: 1.0
testcase: 2924,[0.20239125 0.7966573  0.00095146 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 2925,[0.84476792 0.15235291 0.00287917 0.001     ], SUM[:3]: 1.0
testcase: 2926,[0.50440981 0.49499276 0.00059743 0.001     ], SUM[:3]: 1.0
testcase: 2927,[0.29431955 0.70485299 0.00082747 0.001     ], SUM[:3]: 1.0
testcase: 2928,[0.55623832 0.44270905 0.00105262 0.001     ], SUM[:3]: 1.0
testcase: 2929,[0.0021477  0.99679462 0.00105768 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 2930,[0.14146598 0.85750203 0.00103199 0.001     ], SUM[:3]: 1.0
testcase: 2931,[0.63825177 0.36074281 0.00100542 0.001     ], SUM[:3]: 1.0
testcase: 2932,[0.48914405 0.51027612 0.

testcase: 3387,[0.85642208 0.14260343 0.00097449 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 3388,[0.99242887 0.00632156 0.00124957 0.001     ], SUM[:3]: 1.0
testcase: 3389,[0.67886328 0.31898122 0.0021555  0.001     ], SUM[:3]: 1.0
testcase: 3390,[0.45536409 0.54366651 0.0009694  0.001     ], SUM[:3]: 1.0
testcase: 3391,[0.1169342  0.88147685 0.00158895 0.001     ], SUM[:3]: 1.0
testcase: 3392,[0.88575368 0.11333153 0.00091479 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 3393,[0.6013772  0.39767127 0.00095153 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 3394,[0.62692077 0.37232001 0.00075922 0.001     ], SUM[:3]: 1.0
testcase: 3395,[0.42071046 0.57868127 0.00060827 0.001     ], SUM[:3]: 1.0
testcase: 3396,[0.74569956 0.25255295 0.00174749 0.001     ], SUM[:3]: 1.0
testcase: 3397,[0.85263775 0.14620938 0.00115287 0.001     ], SUM[:3]: 1.0
testcase: 3398,[0.2941158  0.70517129 0.00071292 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 3399,[0.68513484 0.31416406 0.

testcase: 3829,[0.31959534 0.67958123 0.00082343 0.001     ], SUM[:3]: 1.0
testcase: 3830,[0.01537362 0.98280443 0.00182195 0.001     ], SUM[:3]: 1.0
testcase: 3831,[0.61315222 0.38618815 0.00065963 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 3832,[0.39824816 0.60106908 0.00068276 0.001     ], SUM[:3]: 1.0
testcase: 3833,[0.52233272 0.47673065 0.00093663 0.001     ], SUM[:3]: 1.0
testcase: 3834,[0.409193   0.59019702 0.00060999 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 3835,[0.68702451 0.31225585 0.00071963 0.001     ], SUM[:3]: 1.0
testcase: 3836,[0.20989741 0.7869691  0.00313349 0.001     ], SUM[:3]: 1.0
testcase: 3837,[0.65899427 0.34024585 0.00075989 0.001     ], SUM[:3]: 1.0
testcase: 3838,[0.83871216 0.15723338 0.00405446 0.001     ], SUM[:3]: 1.0
testcase: 3839,[0.51187367 0.48756237 0.00056396 0.001     ], SUM[:3]: 1.0
testcase: 3840,[0.97014903 0.02599049 0.00386049 0.001     ], SUM[:3]: 1.0
testcase: 3841,[0.79633324 0.20243016 0.0012366  0.001     ], SUM[:3]:

testcase: 4260,[0.50899449 0.49043005 0.00057546 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 4261,[0.83045787 0.1686344  0.00090773 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 4262,[0.45235669 0.54677622 0.0008671  0.001     ], SUM[:3]: 1.0
testcase: 4263,[0.40699079 0.5920666  0.00094261 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 4264,[0.73211324 0.26704455 0.00084221 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 4265,[0.57393224 0.4224893  0.00357846 0.001     ], SUM[:3]: 1.0
testcase: 4266,[0.58882831 0.40884468 0.00232701 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 4267,[0.02111616 0.97778376 0.00110008 0.001     ], SUM[:3]: 0.9999999999999999
testcase: 4268,[0.58147559 0.41719257 0.00133184 0.001     ], SUM[:3]: 1.0
testcase: 4269,[0.71650546 0.28182493 0.0016696  0.001     ], SUM[:3]: 0.9999999999999999
testcase: 4270,[0.47812553 0.52098241 0.00089206 0.001     ], SUM[:3]: 1.0
testcase: 4271,[0.5616293 0.4375992 0.0007715 0.001    ], SUM[:3]: 1.0

testcase: 4704,[0.45367543 0.54533755 0.00098702 0.001     ], SUM[:3]: 1.0
testcase: 4705,[0.5911506  0.40795075 0.00089865 0.001     ], SUM[:3]: 1.0
testcase: 4706,[0.50081902 0.49763539 0.0015456  0.001     ], SUM[:3]: 1.0
testcase: 4707,[0.92211329 0.07627437 0.00161234 0.001     ], SUM[:3]: 1.0
testcase: 4708,[0.19618506 0.80176389 0.00205106 0.001     ], SUM[:3]: 1.0
testcase: 4709,[0.85272849 0.14565618 0.00161532 0.001     ], SUM[:3]: 1.0
testcase: 4710,[0.79195992 0.18643699 0.02160309 0.001     ], SUM[:3]: 1.0
testcase: 4711,[0.28223546 0.71700335 0.00076119 0.001     ], SUM[:3]: 1.0000000000000002
testcase: 4712,[0.47336671 0.52609801 0.00053529 0.001     ], SUM[:3]: 1.0
testcase: 4713,[0.06188996 0.93679624 0.0013138  0.001     ], SUM[:3]: 1.0
testcase: 4714,[0.35994415 0.63929146 0.00076439 0.001     ], SUM[:3]: 1.0
testcase: 4715,[0.92303895 0.07564726 0.00131379 0.001     ], SUM[:3]: 1.0
testcase: 4716,[2.52693907e-04 9.96810080e-01 2.93722653e-03 1.00000000e-03], SUM[:3]

In [81]:
with open('./results_relaxed.txt', 'w') as result_file_relaxed:
    c = {k: v for k, v in sorted(test_dictionary_relaxed.items(), key=lambda item: sum(item[1]), reverse=True)}
    for key, v in c.items():
        k = list(key[1])
        p = 4
        line = f'testcase: {key[0]:<4}[l3: {np.round(k[0], 4):<6}][l2: {np.round(k[1], 4):<6}][l1: {np.round(k[2], 4):<6}][e: {np.round(k[3], 4):<6}] ==> [positive accuracy: {np.round(v[0], 4):<6}][negative accuracy: {np.round(v[1], 4):<6}]'
        print(line)
        result_file_relaxed.write(line + '\n')

testcase: 138 [l3: 0.3248][l2: 0.6743][l1: 0.0009][e: 0.001 ] ==> [positive accuracy: 0.7899][negative accuracy: 0.7373]
testcase: 244 [l3: 0.3306][l2: 0.6686][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7899][negative accuracy: 0.7373]
testcase: 667 [l3: 0.336 ][l2: 0.6633][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7899][negative accuracy: 0.7373]
testcase: 699 [l3: 0.3369][l2: 0.6623][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7899][negative accuracy: 0.7373]
testcase: 867 [l3: 0.3306][l2: 0.6685][l1: 0.0009][e: 0.001 ] ==> [positive accuracy: 0.7899][negative accuracy: 0.7373]
testcase: 949 [l3: 0.3365][l2: 0.6627][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7899][negative accuracy: 0.7373]
testcase: 965 [l3: 0.3437][l2: 0.6557][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7899][negative accuracy: 0.7373]
testcase: 1008[l3: 0.3194][l2: 0.6796][l1: 0.001 ][e: 0.001 ] ==> [positive accuracy: 0.7899][negative accuracy: 0.7373]
testcase: 1050[l3: 0.3411][l2: 0

testcase: 1091[l3: 0.3799][l2: 0.6194][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7336]
testcase: 1155[l3: 0.3638][l2: 0.6354][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7336]
testcase: 1163[l3: 0.217 ][l2: 0.7793][l1: 0.0037][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7336]
testcase: 1233[l3: 0.3777][l2: 0.6215][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7336]
testcase: 1279[l3: 0.3821][l2: 0.6172][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7336]
testcase: 1285[l3: 0.155 ][l2: 0.8416][l1: 0.0034][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7336]
testcase: 1383[l3: 0.3823][l2: 0.6168][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7336]
testcase: 1393[l3: 0.3823][l2: 0.617 ][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7336]
testcase: 1491[l3: 0.3884][l2: 0

testcase: 1609[l3: 0.5014][l2: 0.4978][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7336]
testcase: 1611[l3: 0.4833][l2: 0.5161][l1: 0.0006][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7355]
testcase: 1621[l3: 0.591 ][l2: 0.4082][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7298]
testcase: 1634[l3: 0.5078][l2: 0.4914][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7336]
testcase: 1635[l3: 0.5034][l2: 0.4961][l1: 0.0005][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7336]
testcase: 1641[l3: 0.5027][l2: 0.4968][l1: 0.0005][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7336]
testcase: 1682[l3: 0.535 ][l2: 0.4642][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7842][negative accuracy: 0.7317]
testcase: 1697[l3: 0.5906][l2: 0.4087][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7298]
testcase: 1701[l3: 0.5654][l2: 0

testcase: 1605[l3: 0.2293][l2: 0.7697][l1: 0.001 ][e: 0.001 ] ==> [positive accuracy: 0.7842][negative accuracy: 0.7298]
testcase: 1607[l3: 0.4508][l2: 0.5487][l1: 0.0005][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7336]
testcase: 1622[l3: 0.5657][l2: 0.4337][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7842][negative accuracy: 0.7298]
testcase: 1647[l3: 0.5544][l2: 0.445 ][l1: 0.0006][e: 0.001 ] ==> [positive accuracy: 0.7842][negative accuracy: 0.7298]
testcase: 1648[l3: 0.5181][l2: 0.4812][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7336]
testcase: 1654[l3: 0.5168][l2: 0.4825][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7336]
testcase: 1670[l3: 0.469 ][l2: 0.5304][l1: 0.0006][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7336]
testcase: 1685[l3: 0.4672][l2: 0.5322][l1: 0.0006][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7336]
testcase: 1690[l3: 0.5772][l2: 0

testcase: 2192[l3: 0.6366][l2: 0.3627][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.728 ]
testcase: 2202[l3: 0.514 ][l2: 0.485 ][l1: 0.001 ][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7317]
testcase: 2203[l3: 0.4565][l2: 0.5425][l1: 0.001 ][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7317]
testcase: 2235[l3: 0.4053][l2: 0.5941][l1: 0.0006][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7317]
testcase: 2239[l3: 0.4798][l2: 0.5185][l1: 0.0017][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7317]
testcase: 2246[l3: 0.4431][l2: 0.5562][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7317]
testcase: 2257[l3: 0.414 ][l2: 0.5854][l1: 0.0006][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7317]
testcase: 2274[l3: 0.5309][l2: 0.4684][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7317]
testcase: 2282[l3: 0.4951][l2: 0

testcase: 3953[l3: 0.6168][l2: 0.3822][l1: 0.001 ][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7261]
testcase: 3968[l3: 0.1672][l2: 0.8313][l1: 0.0015][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7298]
testcase: 3970[l3: 0.2011][l2: 0.7981][l1: 0.0009][e: 0.001 ] ==> [positive accuracy: 0.7824][negative accuracy: 0.7298]
testcase: 3974[l3: 0.6074][l2: 0.391 ][l1: 0.0016][e: 0.001 ] ==> [positive accuracy: 0.7842][negative accuracy: 0.728 ]
testcase: 4000[l3: 0.4655][l2: 0.5337][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7317]
testcase: 4004[l3: 0.6375][l2: 0.3612][l1: 0.0013][e: 0.001 ] ==> [positive accuracy: 0.788 ][negative accuracy: 0.7242]
testcase: 4035[l3: 0.0367][l2: 0.961 ][l1: 0.0023][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7317]
testcase: 4038[l3: 0.0972][l2: 0.9013][l1: 0.0014][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7317]
testcase: 4061[l3: 0.624 ][l2: 0

testcase: 4594[l3: 0.6997][l2: 0.2996][l1: 0.0007][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7205]
testcase: 4610[l3: 0.8692][l2: 0.1298][l1: 0.001 ][e: 0.001 ] ==> [positive accuracy: 0.7749][negative accuracy: 0.7317]
testcase: 4626[l3: 0.7345][l2: 0.2646][l1: 0.0009][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7205]
testcase: 4664[l3: 0.7018][l2: 0.2974][l1: 0.0009][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7205]
testcase: 4682[l3: 0.8696][l2: 0.1294][l1: 0.001 ][e: 0.001 ] ==> [positive accuracy: 0.7749][negative accuracy: 0.7317]
testcase: 4685[l3: 0.6694][l2: 0.3286][l1: 0.002 ][e: 0.001 ] ==> [positive accuracy: 0.7842][negative accuracy: 0.7223]
testcase: 4692[l3: 0.6968][l2: 0.302 ][l1: 0.0013][e: 0.001 ] ==> [positive accuracy: 0.7861][negative accuracy: 0.7205]
testcase: 4731[l3: 0.7066][l2: 0.2923][l1: 0.0011][e: 0.001 ] ==> [positive accuracy: 0.7842][negative accuracy: 0.7223]
testcase: 4739[l3: 0.6573][l2: 0

testcase: 3607[l3: 0.0574][l2: 0.9413][l1: 0.0012][e: 0.001 ] ==> [positive accuracy: 0.7749][negative accuracy: 0.7261]
testcase: 3623[l3: 0.7364][l2: 0.2623][l1: 0.0013][e: 0.001 ] ==> [positive accuracy: 0.7805][negative accuracy: 0.7205]
testcase: 3696[l3: 0.8965][l2: 0.1023][l1: 0.0012][e: 0.001 ] ==> [positive accuracy: 0.7749][negative accuracy: 0.7261]
testcase: 3734[l3: 0.8992][l2: 0.0996][l1: 0.0012][e: 0.001 ] ==> [positive accuracy: 0.7767][negative accuracy: 0.7242]
testcase: 3789[l3: 0.0699][l2: 0.9292][l1: 0.001 ][e: 0.001 ] ==> [positive accuracy: 0.7749][negative accuracy: 0.7261]
testcase: 3848[l3: 0.0382][l2: 0.9606][l1: 0.0012][e: 0.001 ] ==> [positive accuracy: 0.7711][negative accuracy: 0.7298]
testcase: 3875[l3: 0.1097][l2: 0.8892][l1: 0.0011][e: 0.001 ] ==> [positive accuracy: 0.7749][negative accuracy: 0.7261]
testcase: 3952[l3: 0.0609][l2: 0.9376][l1: 0.0015][e: 0.001 ] ==> [positive accuracy: 0.7767][negative accuracy: 0.7242]
testcase: 3964[l3: 0.1143][l2: 0

testcase: 3978[l3: 0.7789][l2: 0.2197][l1: 0.0013][e: 0.001 ] ==> [positive accuracy: 0.7767][negative accuracy: 0.7167]
testcase: 4069[l3: 0.9194][l2: 0.0768][l1: 0.0038][e: 0.001 ] ==> [positive accuracy: 0.7636][negative accuracy: 0.7298]
testcase: 4217[l3: 0.8104][l2: 0.1883][l1: 0.0013][e: 0.001 ] ==> [positive accuracy: 0.7711][negative accuracy: 0.7223]
testcase: 4302[l3: 0.9089][l2: 0.0896][l1: 0.0015][e: 0.001 ] ==> [positive accuracy: 0.773 ][negative accuracy: 0.7205]
testcase: 4320[l3: 0.7796][l2: 0.219 ][l1: 0.0014][e: 0.001 ] ==> [positive accuracy: 0.7749][negative accuracy: 0.7186]
testcase: 4332[l3: 0.8433][l2: 0.1558][l1: 0.0009][e: 0.001 ] ==> [positive accuracy: 0.7692][negative accuracy: 0.7242]
testcase: 4404[l3: 0.815 ][l2: 0.1838][l1: 0.0013][e: 0.001 ] ==> [positive accuracy: 0.7711][negative accuracy: 0.7223]
testcase: 4440[l3: 0.8288][l2: 0.1704][l1: 0.0008][e: 0.001 ] ==> [positive accuracy: 0.7711][negative accuracy: 0.7223]
testcase: 4441[l3: 0.9191][l2: 0

testcase: 1404[l3: 0.9715][l2: 0.0271][l1: 0.0015][e: 0.001 ] ==> [positive accuracy: 0.7523][negative accuracy: 0.7148]
testcase: 2049[l3: 0.9708][l2: 0.0274][l1: 0.0019][e: 0.001 ] ==> [positive accuracy: 0.7505][negative accuracy: 0.7167]
testcase: 2118[l3: 0.971 ][l2: 0.0274][l1: 0.0015][e: 0.001 ] ==> [positive accuracy: 0.7523][negative accuracy: 0.7148]
testcase: 3779[l3: 0.9744][l2: 0.0223][l1: 0.0033][e: 0.001 ] ==> [positive accuracy: 0.7486][negative accuracy: 0.7186]
testcase: 2790[l3: 0.9776][l2: 0.0194][l1: 0.003 ][e: 0.001 ] ==> [positive accuracy: 0.7486][negative accuracy: 0.7167]
testcase: 852 [l3: 0.9753][l2: 0.0222][l1: 0.0025][e: 0.001 ] ==> [positive accuracy: 0.7505][negative accuracy: 0.7148]
testcase: 1362[l3: 0.9688][l2: 0.0291][l1: 0.0021][e: 0.001 ] ==> [positive accuracy: 0.7467][negative accuracy: 0.7186]
testcase: 3840[l3: 0.9701][l2: 0.026 ][l1: 0.0039][e: 0.001 ] ==> [positive accuracy: 0.7505][negative accuracy: 0.7148]
testcase: 1276[l3: 0.9668][l2: 0